# Marketing Mix Modeling - Exemplo Completo

Este notebook demonstra o fluxo completo do MMM, desde a ingestao de dados ate a otimizacao de budget.


In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import plotly.express as px
from loguru import logger


## 1. Ingestao de Dados

Geramos dados sinteticos para demonstracao. Em producao, carregue dados reais via CSV ou Excel.


In [ ]:
from src.ingestion.loader import DataLoader
from src.ingestion.validator import DataValidator

loader = DataLoader()
df = loader.generate_sample_data(n_weeks=104)

validator = DataValidator()
is_valid, issues = validator.validate(df)
print(f"Dados validos: {is_valid}")
if issues:
    for issue in issues:
        print(f"  - {issue}")

df.head(10)

## 2. Feature Engineering

Aplicamos transformacoes de adstock e saturacao nos canais de midia.


In [ ]:
from src.features.adstock import AdstockTransformer
from src.features.saturation import SaturationTransformer
from src.features.engineering import FeatureEngineer
from src.config import MEDIA_CHANNELS

adstock = AdstockTransformer(method="geometric")
df_transformed = adstock.transform(df)

saturation = SaturationTransformer(method="exponential")
df_transformed = saturation.transform(df_transformed)

engineer = FeatureEngineer()
df_features = engineer.create_date_features(df_transformed)
df_features = df_features.dropna().reset_index(drop=True)

print(f"Shape final: {df_features.shape}")
df_features.columns.tolist()

## 3. Treinamento do Modelo

Comparamos tres tipos de regressao regularizada.


In [ ]:
from src.models.regularized import RegularizedMMM, compare_models

feature_cols = [
    c for c in df_features.select_dtypes(include=[np.number]).columns
    if c not in ("revenue", "date")
]

comparison = compare_models(df_features, feature_cols)
print(comparison.to_string(index=False))

In [ ]:
mmm = RegularizedMMM(model_type="ridge", alpha=1.0)
X_train, X_test, y_train, y_test = mmm.prepare_data(df_features, feature_cols)
mmm.fit(X_train, y_train)
metrics = mmm.evaluate(X_test, y_test)

print("Metricas do modelo:")
for k, v in metrics.items():
    print(f"  {k}: {v:.4f}")

## 4. Analise de Importancia


In [ ]:
from src.models.importance import FeatureImportanceAnalyzer

analyzer = FeatureImportanceAnalyzer(feature_names=feature_cols)
importance = analyzer.coefficient_importance(mmm.model)
importance.head(15)

## 5. Visualizacoes


In [ ]:
from src.visualization.charts import MMMCharts

charts = MMMCharts()

y_pred = mmm.predict(df_features[feature_cols].values)
fig = charts.plot_actual_vs_predicted(
    df_features["date"] if "date" in df_features.columns else pd.Series(range(len(df_features))),
    df_features["revenue"].values,
    y_pred,
)
fig.show()

In [ ]:
coefs = mmm.get_coefficients()
media_coefs = coefs[coefs["feature"].str.contains("spend")].head(10)

fig = charts.plot_roi_comparison(
    pd.DataFrame({
        "channel": media_coefs["feature"].str.replace("_spend", ""),
        "roas": media_coefs["abs_coefficient"],
    })
)
fig.show()

## 6. Otimizacao de Budget


In [ ]:
from src.optimization.budget import BudgetOptimizer

base_features = df_features[feature_cols].mean().values
media_in_features = [c for c in MEDIA_CHANNELS if c in feature_cols]

optimizer = BudgetOptimizer(
    model=mmm.model,
    scaler=mmm.scaler,
    feature_names=feature_cols,
    media_channels=media_in_features,
)

result = optimizer.optimize(total_budget=500000, base_features=base_features)

print(f"Receita prevista: R$ {result["predicted_revenue"]:,.0f}")
print(f"Melhoria: {result["improvement_pct"]:.2f}%")
print("
Alocacao otima:")
for ch, val in result["optimal_allocation"].items():
    print(f"  {ch}: R$ {val:,.0f}")

## 7. Cenarios


In [ ]:
from src.optimization.scenarios import ScenarioPlanner

planner = ScenarioPlanner(
    model=mmm.model,
    scaler=mmm.scaler,
    feature_names=feature_cols,
    media_channels=media_in_features,
)

planner.create_baseline(base_features)
planner.create_proportional_scenario(base_features, 0.5, "Corte 50%")
planner.create_proportional_scenario(base_features, 1.5, "Aumento 50%")

if len(media_in_features) >= 2:
    planner.create_reallocation_scenario(
        base_features,
        from_channel=media_in_features[0],
        to_channel=media_in_features[1],
        transfer_pct=0.3,
    )

comparison = planner.compare_scenarios()
comparison

---

*"O conhecimento fala, mas a sabedoria escuta." - Jimi Hendrix*
